In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import functions_PgSql
import psycopg2
import datetime
import folium
import os
from matplotlib import pyplot as plt

In [2]:
# Spécifier le nom de la base de données créée au préalable dans PgAdmin
ma_base_donnees = "Musees_V3"
utilisateur = "postgres"

# Méthode os.environ.get pour utiliser le mot de passe 
# enregistré au préalable dans une variable d'environnement
mot_passe = os.environ.get('pg_psw')

# Fonction conn appelle la fonction ouvrir_connexion du fichier annexe Python
conn = functions_PgSql.ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

cursor = conn.cursor()

## Graph 1 : Nombre d'oeuvres par villes

In [3]:

# Graphique 1 : 
# Nombre d'oeuvres et musées par villes, filtre sur >1000 oeuvres exposées
oeuvre_villes_musees = pd.read_sql_query('''
                        SELECT m.ville, 
                               COUNT(o.id_oeuvre) AS Total_Oeuvres,
                               COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
                        FROM oeuvre o, musee m
                        WHERE o.Id_Museo = m.Id_Museo
                        GROUP BY ville
                        HAVING COUNT(o.id_oeuvre) > 1000
                        ORDER BY Total_Oeuvres DESC
                        LIMIT 15;''',conn)


In [4]:
fig = px.bar(oeuvre_villes_musees, x='ville', y='total_oeuvres',
             hover_data=['nb_musees', 'total_oeuvres'], color='nb_musees',
             labels={'Nb':'Nb Oeuvres'}, height=400, width=1000)
fig.show()

## Graph 2 : Nombre de musées par villes

In [5]:
# Nombre d'oeuvres et musées par villes, filtre sur >1000 oeuvres exposées
oeuvre_musees_villes = pd.read_sql_query('''
                        SELECT m.ville, 
                               COUNT(o.id_oeuvre) AS Total_Oeuvres,
                               COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
                        FROM oeuvre o, musee m
                        WHERE o.Id_Museo = m.Id_Museo
                        GROUP BY ville
                        HAVING COUNT(o.id_oeuvre) > 1000
                        ORDER BY Nb_Musees DESC
                        LIMIT 15;''',conn)


In [6]:
fig = px.bar(oeuvre_musees_villes, x='ville', y='nb_musees',
             hover_data=['nb_musees', 'total_oeuvres'], color='total_oeuvres',
             labels={'Nb':'Nb Oeuvres'}, height=400, width=1000)
fig.show()

## Graph 3 : Nombre de musées et oeuvres par villes

In [7]:
fig = go.Figure(data=[
    go.Bar(name='Musées', x=oeuvre_villes_musees.ville, y=oeuvre_villes_musees.nb_musees),
    go.Bar(name='Oeuvres', x=oeuvre_villes_musees.ville, y=oeuvre_villes_musees.total_oeuvres)
])
# Change the bar mode
#fig.update_layout(barmode='group')
fig.show()

## Graph 4 : Nombre de musées par régions

In [8]:
# Nombre d'oeuvres et musées par régions LEONARD DE VINCI
oeuvre_region = pd.read_sql_query('''SELECT r.nom_region,
                                COUNT(DISTINCT oa.id_museo) AS Nb_Musees,
                                COUNT( DISTINCT oa.id_oeuvre) AS Total_Oeuvres
                                FROM musee m, departement d, region r, 
                                v_Oeuvres_Artistes oa 
                                WHERE oa.Id_Museo = m.Id_Museo 
                                AND m.code_dpt = d.id_dpt 
                                AND d.id_region = r.id_region 
                                AND oa.nom_artiste LIKE '%VINCI%'
                                GROUP BY nom_region 
                                ORDER BY total_oeuvres DESC;''',conn)

In [9]:
fig = px.bar(oeuvre_region, x='nom_region', y='nb_musees',
             hover_data=['total_oeuvres','nb_musees'], color='total_oeuvres',
             labels={'Nb':'Nb Oeuvres'}, height=400, width=1000)
fig.show()

In [10]:
df = pd.read_sql_query('''SELECT m.ville, 
                                COUNT(o.id_oeuvre) AS Total_Oeuvres, 
                                ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) 
                          FROM oeuvre o),2) AS Part_Oeuvres 
                          FROM oeuvre o, musee m 
                          WHERE o.Id_Museo = m.Id_Museo 
                          GROUP BY ville''',conn)

In [11]:
df.head()

,ville,total_oeuvres,part_oeuvres
0,Abbeville,132,0.03
1,Agde,3,0.00
2,Agen,24,0.01
3,Aix-en-Provence,14,0.00
4,Aix-les-Bains,1,0.00


In [12]:
df.shape

(296, 3)

In [13]:
# Total oeuvres par domaine et musées
# Par domaine et villes
# Recherche par ville et/ou domaine

requete = """
    SELECT m.ville,
           m.nom_officiel,
           oc.Nom_Categorie,
           COUNT(oc.titre_oeuvre) AS Total_Oeuvres,
           COUNT(DISTINCT oc.id_museo) AS Nb_Musees,
           m.geo_x,
           m.geo_y
    FROM v_Oeuvres_Categories oc
 	
    JOIN musee m
        ON oc.id_museo = m.id_museo

    WHERE geo_x != 'Nan' AND geo_y != 'Nan' 

    GROUP BY Nom_Categorie, m.ville, m.nom_officiel, m.geo_x, m.geo_y;
"""
Categories_villes = pd.read_sql_query(requete, conn)
print(Categories_villes.shape)
Categories_villes.head()

(3500, 7)


,ville,nom_officiel,nom_categorie,total_oeuvres,nb_musees,geo_x,geo_y
0,Barcelonnette,musée de la vallée,AFRIQUE,2,1,44.387609,6.655077
1,Beauvais,MUDO - musée de l'Oise,AFRIQUE,1,1,49.432788,2.080444
2,Belfort,musée d'Art et d'Histoire,AFRIQUE,1,1,47.637570,6.867072
3,Besançon cedex,muséum d'histoire naturelle,AFRIQUE,36,1,47.236687,6.022095
4,Bordeaux,musée d'Aquitaine,AFRIQUE,2,1,44.835543,-0.575205


## Map 1 : Situer les musées exposant un artiste

In [14]:
# Total oeuvres par domaine et musées
# Par domaine et villes
# Recherche par ville et/ou domaine

requete = """
    SELECT m.ville,
           m.nom_officiel,
           oa.nom_artiste,
           COUNT(oa.id_oeuvre) AS Total_Oeuvres,
           m.geo_x,
           m.geo_y
    FROM v_Oeuvres_Artistes oa
 	
    JOIN musee m
        ON oa.id_museo = m.id_museo
  
    WHERE geo_x != 'Nan' AND geo_y != 'Nan' 
    AND Oa.nom_artiste LIKE '%CALDER ALEX%'

    GROUP BY nom_artiste, m.ville, m.nom_officiel, m.geo_x, m.geo_y;
"""
Artistes_villes = pd.read_sql_query(requete, conn)
print(Artistes_villes.shape)
Artistes_villes.head(10)

(5, 6)


,ville,nom_officiel,nom_artiste,total_oeuvres,geo_x,geo_y
0,Antibes,musée Picasso,CALDER ALEXANDER,1,43.580973,7.127959
1,Grenoble,musée de Grenoble,CALDER ALEXANDER,3,45.194026,5.732094
2,Rennes,musée des Beaux-Arts,CALDER ALEXANDER,1,48.109837,-1.674981
3,Sèvres,Sèvres – Cité de la Céramique,CALDER ALEXANDER,4,48.817037,2.205733
4,Tournus,musée Greuze - hôtel-Dieu,CALDER ALEXANDER,1,46.562137,4.910839


In [15]:
MUSEES = Artistes_villes['ville']
LATS = Artistes_villes['geo_y']
LONGS = Artistes_villes['geo_x']
NOM = Artistes_villes['nom_officiel']
ARTISTE = Artistes_villes['nom_artiste']
NB_OEUVRES = Artistes_villes['total_oeuvres']

map = folium.Map(tiles='OpenStreetMap', zoom_start=13, zoom_control=True, control_scale=True)

for i in range(len(MUSEES)):
    folium.Marker(
        location = (LONGS[i], LATS[i]),
        popup = (str(ARTISTE[i]) + ' ,' + str(NOM[i]) + ' ,' + str(NB_OEUVRES[i]) + ' Oeuvre(s) exposée(s)'),
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(map)
map.fit_bounds(map.get_bounds())
fig = map
fig
# tiles = 'cartodbpositron'
# tiles='Stamen Terrain'

In [16]:
# fig.save("Calder.html")

## Map 2 : Situer les oeuvres selon le siècle de création

In [17]:
# Quels Musées Exposent un Artiste sur une période donnée
# Recherche par période : siècle, Infos Musées et noms artistes et nombre d'oeuvres exposées
# Utilisation de la vue v_Oeuvres_Artistes avec une autre jointure
requete = """
    SELECT COUNT(a.id_oeuvre) AS Nb_Oeuvres,
           m.nom_officiel AS Nom_Musee,
           m.ville,
           m.geo_x,
           m.geo_y
    FROM v_Oeuvres_Artistes a
    
    JOIN musee m 
        ON a.Id_Museo = m.Id_Museo
    
    WHERE  a.Periode_Oeuvre LIKE '%21%'
    AND geo_x != 'Nan' AND geo_y != 'Nan' 
        
    GROUP BY m.nom_officiel, 
             m.ville,
             m.geo_x,
             m.geo_y

    HAVING COUNT(a.id_oeuvre) > 10
    ORDER BY ville;
"""
Localiser_Periode = pd.read_sql_query(requete, conn)
print(Localiser_Periode.shape)
Localiser_Periode.head()

(26, 5)


,nb_oeuvres,nom_musee,ville,geo_x,geo_y
0,12,musée Rolin,Autun,46.945723,4.299216
1,292,musée des Beaux-Arts et d'Archéologie,Besançon,47.240055,6.022938
2,12,musée d'Art Moderne,Céret,42.485799,2.748490
3,14,musée Arthur Rimbaud,Charleville-Mézières,49.776034,4.720613
4,18,Château-musée,Dieppe,49.923908,1.071756


In [18]:
MUSEES = Localiser_Periode['ville']
LATS = Localiser_Periode['geo_y']
LONGS = Localiser_Periode['geo_x']
NOM = Localiser_Periode['nom_musee']
NB_OEUVRES = Localiser_Periode['nb_oeuvres']

map = folium.Map(tiles='OpenStreetMap', zoom_start=4, zoom_control=True, control_scale=True, title= 'Lieu d\'expostion des oeuvres du 21e siècle')

for i in range(len(MUSEES)):
    folium.Marker(
        location = (LONGS[i], LATS[i]),
        popup = (str(NOM[i]) + ' ,' + str(NB_OEUVRES[i]) + ' Oeuvre(s) exposée(s)'),
        icon=folium.Icon(color="lightred", icon="info-sign")
    ).add_to(map)
map.fit_bounds(map.get_bounds())
fig = map
fig

In [19]:
# fig.save("21eme.html")

## Map 3 : Situer les oeuvres selon leur catégorie

In [20]:
# Total oeuvres par domaine et musées
# Par domaine et villes
# Recherche par ville et/ou domaine

requete = """
            SELECT oc.nom_categorie,
            m.ville,
            m.nom_officiel as nom_musee,
            COUNT(oc.id_oeuvre) as Nombre_Oeuvres,
            COUNT(distinct oc.id_museo) as Nombre_Musees,
            m.geo_x,
            m.geo_y

        FROM v_Oeuvres_Categories oc

        JOIN musee m
                ON oc.id_museo = m.id_museo
        
            WHERE geo_x != 'Nan' AND geo_y != 'Nan' 
            AND oc.nom_categorie LIKE '%ROMAIN%'
            GROUP BY oc.nom_categorie, m.ville, m.nom_officiel, m.geo_x, m.geo_y;
"""
Categories_geo = pd.read_sql_query(requete, conn)

In [21]:
MUSEES = Categories_geo['ville']
LATS = Categories_geo['geo_y']
LONGS = Categories_geo['geo_x']
NOM = Categories_geo['nom_musee']
CATEGORIE = Categories_geo['nom_categorie']
NB_OEUVRES = Categories_geo['nombre_oeuvres']

map = folium.Map(tiles='OpenStreetMap', zoom_start=4, zoom_control=True, control_scale=True, title= 'Lieu d\'expostion par domaine')

for i in range(len(MUSEES)):
    folium.Marker(
        location = (LONGS[i], LATS[i]),
        popup = (str(CATEGORIE[i]) + ' ,' + str(NOM[i]) + ' ,' + str(NB_OEUVRES[i]) + ' Oeuvre(s) exposée(s)'),
        icon=folium.Icon(color="lightred", icon="info-sign")
    ).add_to(map)
map.fit_bounds(map.get_bounds())
fig = map
fig

In [22]:
# fig.save("ROMAIN.html")

## Graph 5 : Le nombre d'acquisitions par années

In [23]:
# Dates d'acquisition, nombre d'oeuvres
requete = """
    SELECT o.date_acquisition,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) FROM oeuvre o),2) || ' %' AS Part_Oeuvres,
           COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
           
    FROM oeuvre o,
    	musee m
 	
    WHERE o.Id_Museo = m.Id_Museo
    AND date_acquisition != 'NaN'
 
    GROUP BY date_acquisition
    HAVING COUNT(o.id_oeuvre) > 10
    ORDER BY date_acquisition DESC;
"""
date_acquisition = pd.read_sql_query(requete, conn)
print(date_acquisition.shape)
date_acquisition.head()

(536, 4)


,date_acquisition,total_oeuvres,part_oeuvres,nb_musees
0,4-2003,55,0.01 %,1
1,30-07-1992,126,0.03 %,1
2,29-09-2016,12,0.00 %,1
3,27-05-2014,63,0.02 %,1
4,26-07-1974,22,0.01 %,1


In [24]:

date_acquisition.date_acquisition = date_acquisition.date_acquisition.replace(regex=True, to_replace= (r'[^0-9/-]', r'[^0-9.](-)*', r'[0-9]{2}-', r'[0-9]{1}-', r'-[0-9]{2}-', r'-[0-9]{1}-', r'[0-9]{1}-'), value=(r'-', r'-', r'-', r'-', r'-', r'-', r'-')).str.strip('-')

In [25]:
date_acquisition = date_acquisition.sort_values(by='date_acquisition')

In [26]:
mask = date_acquisition.date_acquisition.isin(['29', '30', '31', '22', '16', '19', '26', '27', '28', '15', '14', '12', '11', '10', '9', '17', '18', '23', '25', '09', '07', '06', '05', '04', '03', '02', '01', '21', '08', '1', '20'])
date_acquisition = date_acquisition[~mask]


In [27]:
fig5 = px.area(x=date_acquisition.date_acquisition, y=date_acquisition.total_oeuvres, title = 'Nombre d\'oeuvres acquises par année')
fig5.update_layout(
     title_font_size = 40, 
     width = 1000, height = 600)
fig5.update_xaxes(
     title_text = 'Années',
     title_font=dict(size=15, family='Verdana', color='black'), 
     tickfont=dict(family='Calibri', color='darkred', size=15))
fig5.update_yaxes(
     title_text = "Nombre Oeuvres",
     title_font=dict(size=15,family='Verdana',color='black'), 
     tickfont=dict(family='Calibri', color='darkred', size=15))

In [28]:
# Dates d'acquisition, nombre d'oeuvres
requete = """
    SELECT o.date_acquisition,
           m.nom_officiel,
           COUNT(o.id_oeuvre) AS Total_Oeuvres,
           ROUND(ROUND(COUNT(o.id_oeuvre),2)*100/(SELECT ROUND(COUNT(o.id_oeuvre),2) FROM oeuvre o),2) || ' %' AS Part_Oeuvres,
           COUNT(DISTINCT o.Id_Museo) AS Nb_Musees
           
    FROM oeuvre o,
    	musee m
 	
    WHERE o.Id_Museo = m.Id_Museo
    AND date_acquisition != 'NaN'
    AND date_acquisition = '2011'
    GROUP BY date_acquisition, nom_officiel
    HAVING COUNT(o.id_oeuvre) > 10
    ORDER BY date_acquisition DESC;
"""
acquisition = pd.read_sql_query(requete, conn)
print(acquisition.shape)
acquisition.head(10)

(6, 5)


,date_acquisition,nom_officiel,total_oeuvres,part_oeuvres,nb_musees
0,2011,"les Abattoirs, musée d'art moderne et contempo...",33,0.01 %,1
1,2011,musée Danicourt,45,0.01 %,1
2,2011,musée de la faïence et des techniques faïencières,277,0.07 %,1
3,2011,musée du Papier Peint,12,0.00 %,1
4,2011,musée Municipal,30,0.01 %,1
5,2011,Sèvres – Cité de la Céramique,17094,4.13 %,1
